In [120]:
import pandas as pd
import numpy as np
import openpyxl
import csv
import sqlite3
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, cross_validate
from numpy import argmax
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.feature_selection import chi2, f_classif, SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



In [121]:
crashes = pd.read_csv("crash_data_2009.csv", dtype=object)

In [122]:
crashes.head()

,case_id,jurisdiction,officer_id,reporting_district,chp_shift,population,county_city_location,county_location,special_condition,beat_type,...,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,primary_ramp,secondary_ramp,latitude,longitude,collision_date,collision_time,process_date
0,3858022,1005,P379,2C,not chp,>250000,1005,fresno,0,not chp,...,0,0,0,NaN,NaN,NaN,NaN,2009-02-03,17:11:00,2009-04-27
1,3899441,9120,17248,NaN,2200 thru 0559,2500 to 10000,0801,del norte,0,chp state highway,...,0,0,0,NaN,NaN,NaN,NaN,2009-02-28,01:45:00,2009-11-02
2,3899442,9530,19005,NaN,0600 thru 1359,>250000,1942,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,33.86465,-118.28533,2009-02-09,10:20:00,2010-01-14
3,3899445,9530,19284,NaN,1400 thru 2159,>250000,1942,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,33.7912,-118.2823,2009-02-18,15:50:00,2010-01-13
4,3899446,9530,19289,NaN,1400 thru 2159,25000 to 50000,1939,los angeles,0,chp state highway,...,0,0,0,NaN,NaN,33.8845,-118.3526,2009-02-11,17:35:00,2010-01-11


# Preprocessing

### drop useless columns

### divide features into sparse features and dense features

#### sparse feature -> one hot encoder

#### dense feature -> Standard Scaler

In [123]:
def monthCol(df, col):
    month = df[col].str.extract(r"^\d{4}\-0(\d)|^\d{4}\-(\d{2})", expand=False).values
    return pd.Series(month[month == month].astype(int))

def hourCol(df, col):
    hour = df[col].str.extract(r"(^[^0]\d)|^0(\d)", expand=False).values
    return pd.Series(hour[hour == hour]).astype(int)

crashes["crash_month"] = monthCol(crashes, "collision_date")
crashes["crash_hour"] = hourCol(crashes, "collision_time")

In [124]:
clean_list = ["county_location", "population", "weather_1", "road_surface", "collision_severity", 
              "lighting", "party_count", "injured_victims", "killed_victims", "crash_hour", 
              "crash_month", "type_of_collision", "hit_and_run", "pedestrian_action", 
              "pedestrian_collision", "pcf_violation_category", "motor_vehicle_involved_with"]

def removeNaN(df, clean_list):
    """
    removes NaN from given features in df
    """
    copy = df.copy()
    return copy.dropna(subset=clean_list).reset_index(drop=True)
crashes = removeNaN(crashes, clean_list)
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403251 entries, 0 to 403250
Data columns (total 77 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   case_id                          403251 non-null  object 
 1   jurisdiction                     402432 non-null  object 
 2   officer_id                       401257 non-null  object 
 3   reporting_district               155928 non-null  object 
 4   chp_shift                        403251 non-null  object 
 5   population                       403251 non-null  object 
 6   county_city_location             403251 non-null  object 
 7   county_location                  403251 non-null  object 
 8   special_condition                403251 non-null  object 
 9   beat_type                        403251 non-null  object 
 10  chp_beat_type                    403251 non-null  object 
 11  city_division_lapd               26036 non-null   object 
 12  ch

In [125]:
numerical_features = ["party_count", "injured_victims", "killed_victims", "crash_hour", "crash_month"]

def convertToInt(df, numerical_features):
    """
    converts numerical_features 
    in df to type int
    """
    copy = df.copy()
    for f in numerical_features:
        if copy[f].dtype != "float64":
            copy[f] = crashes[f].astype(int)
    return copy
crashes = convertToInt(crashes, numerical_features)

In [126]:
def populationConvert(df):
    """
    converts population to state whether
    the area of a crash has >100000 population
    or <100000 population
    """
    copy = df.copy()
    copy["population"] = copy["population"].str.replace("unincorporated", "1000")
    copy_pop = copy["population"].str.extract(r"(\d+)\s\w.+|\>(\d+)|\<(\d+)|[^\>\<](\d+)", expand=False).values
    copy["population"] = pd.Series(copy_pop[copy_pop == copy_pop].astype(int)).apply(lambda num: ">100000" if num>100000 else "<100000")
    return copy
crashes = populationConvert(crashes)

In [127]:
# The features used for classification need to be divided into sparse features and dense features
sparse_feature_list = ["population", "weather_1", "road_surface", "lighting", \
                       "crash_hour", "crash_month", 'hit_and_run', 'pedestrian_action', \
                           "pcf_violation_category", 'motor_vehicle_involved_with']
dense_feature_list = ["party_count"]

# label to be predicted
label_column = 'county_location'

# delete columns from drop_list
crashes.drop([l for l in crashes.columns if l not in sparse_feature_list + dense_feature_list + [label_column]], axis=1, inplace=True)

# preprocess dense feature
for feat in dense_feature_list:
    scaler = MinMaxScaler()
    crashes.loc[:,feat] = scaler.fit_transform(crashes.loc[:, feat].values.reshape(-1, 1))
# preprocess label
lb = LabelEncoder()
crashes.loc[:,label_column] = lb.fit_transform(crashes.loc[:,label_column])
# preprocess sparse feature to one hot feature
crashes = pd.get_dummies(crashes, columns=sparse_feature_list)


In [128]:
# logistic regression
from sklearn.linear_model import LogisticRegression
def myLR(X, Y, val_X, val_Y, max_iter):
    lr = LogisticRegression(C=1000, random_state=0, max_iter=max_iter)
    lr.fit(X, Y)
    return 'lr', accuracy_score(val_Y, lr.predict(val_X))
    
    
    # output = cross_validate(lr, X, Y, cv=10, scoring='accuracy', return_estimator=True)
    # metric = output['test_score']
    # metric.sort()
    # max_acc_estimator = output['estimator'][argmax(output['test_score'])]
    # feature_importances = pd.DataFrame(max_acc_estimator.coef_[0],
    #                                 index = X.columns,
    #                                 columns=['importance']).sort_values('importance', ascending=False)
    # return 'lr', metric, feature_importances

# SVM with linear regression
from sklearn import svm
def mySVM(X, Y, val_X, val_Y, max_iter):
    svc = svm.SVC(C=1.0, kernel='linear', gamma='auto', max_iter=max_iter)
    svc.fit(X, Y)
    return 'svm', accuracy_score(val_Y, svc.predict(val_X))
    # output = cross_validate(svc, X, Y, cv=10, scoring='accuracy', return_estimator=True)
    # metric = output['test_score']
    # metric.sort()
    # max_acc_estimator = output['estimator'][argmax(output['test_score'])]
    # feature_importances = pd.DataFrame(max_acc_estimator.coef_[0],
    #                                     index = X.columns,
    #                                     columns=['importance']).sort_values('importance', ascending=False)
    # return 'svm', metric, feature_importances

# Random Forest
from sklearn.ensemble import RandomForestClassifier
def myRandomForest(X, Y, val_X, val_Y, max_iter):
    RF = RandomForestClassifier(n_estimators=30, criterion='gini', random_state=10)
    RF.fit(X, Y)
    return 'RF', accuracy_score(val_Y, RF.predict(val_X))
    
    # output = cross_validate(RF, X, Y, cv=10, scoring='accuracy', return_estimator=True)
    # metric = output['test_score']
    # metric.sort()
    # max_acc_estimator = output['estimator'][argmax(output['test_score'])]
    # feature_importances = pd.DataFrame(max_acc_estimator.feature_importances_,
    #                                     index = X.columns,
    #                                     columns=['importance']).sort_values('importance', ascending=False)
    # return 'rf', metric, feature_importances
    

    


In [129]:


# NOTE
#   Use chi square test to select the features with the
#   top K scores (- 1 represents all features), and
#   then use these K features for classification
def get_k_important_features(k, rawX, rawY):
    if k == 'all':
        k = len(rawX.columns.values)
    model = SelectKBest(chi2, k=k)
    X = model.fit_transform(rawX, rawY)
    Y = rawY
    scores = model.scores_
    p_values = model.pvalues_
    indices = np.argsort(scores)[::-1]
    k_best_features = list(rawX.columns.values[indices[0:k]])
    # print(f'k={k} best features are: {str(k_best_features)}')
    return k_best_features

MAX_ITER = 1000
rawX = crashes.drop([label_column], axis=1)
rawY = crashes.loc[:, label_column]


In [130]:
K_list = ['all']
classifiers = [myLR, mySVM, myRandomForest]
for k in K_list:
    k_best_features = get_k_important_features(k, rawX, rawY)
    print(f'k={k} best features are: {str(k_best_features)}')
    X = rawX[k_best_features]
    Y = rawY
    X, val_X, Y, val_Y = train_test_split(X, Y, test_size=0.1, random_state=42)
    metric_all = {}
    for classifier in classifiers:
        ret = classifier(X, Y, val_X, val_Y, MAX_ITER)
        feature_importance = None
    
        if len(ret) == 2:
            name, metric = ret
        else:
            name, metric, feature_importance = ret
        if feature_importance is not None:
            print(f'K={k} best important features, the feature importance for method {name} is ')
            print(feature_importance)
        metric_all[name] = metric
        print(metric)
    print(f'------------------------K = {k} best features result------------')
    print(metric_all)

k=all best features are: ['population_>100000', 'road_surface_snowy', 'population_<100000', 'weather_1_snowing', 'lighting_dark with no street lights', 'motor_vehicle_involved_with_non-collision', 'motor_vehicle_involved_with_animal', 'motor_vehicle_involved_with_fixed object', 'road_surface_wet', 'lighting_dark with street lights', 'weather_1_cloudy', 'pcf_violation_category_improper turning', 'motor_vehicle_involved_with_other motor vehicle', 'pcf_violation_category_wrong side of road', 'motor_vehicle_involved_with_parked motor vehicle', 'pcf_violation_category_other than driver (or pedestrian)', 'hit_and_run_misdemeanor', 'motor_vehicle_involved_with_pedestrian', 'pcf_violation_category_unsafe lane change', 'pedestrian_action_crossing in intersection crosswalk', 'pcf_violation_category_following too closely', 'weather_1_fog', 'pcf_violation_category_dui', 'hit_and_run_felony', 'pcf_violation_category_unknown', 'pcf_violation_category_speeding', 'pcf_violation_category_automobile rig

/Users/houpengyu/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/houpengyu/miniforge3/lib/python3.9/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


------------------------K = all best features result------------
{'lr': 0.5999057679908744, 'svm': 0.58227297525169866, 'RF': 0.63858056836780242}
